In [3]:
#external libraries
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point,Polygon
import matplotlib.pyplot as plt
import folium
from folium import plugins
from datetime import datetime
%matplotlib inline
%load_ext autoreload

In [4]:
#local imports
from func.gdfLoader import gdfLoader

In [5]:
MapDataPath="data/cartePalaiseau.csv"
ModuleMapPath="data/carteModulePalaiseau.csv"

Data_Path="data/manip1.1.csv"

sampling_time=2000#ms #si supérieur à 2000ms il faut implémenter la moyenne dans la liste des rssi dans marker dict list



In [6]:
#load the data
map_gdf=gdfLoader(MapDataPath)
#change the projection to longitude /latitude
map_gdf.to_crs(epsg=4326, inplace=True)
map_gdf.head()

,ID_element,nom,geometry
0,21205,4A101,"POLYGON ((2.19977 48.71281, 2.19979 48.71287, ..."
1,21264,4A413,"POLYGON ((2.20045 48.71307, 2.20046 48.71311, ..."
2,21265,4A409,"POLYGON ((2.20043 48.71301, 2.20044 48.71304, ..."
3,21266,,"POLYGON ((2.20050 48.71315, 2.20055 48.71315, ..."
4,21267,,"POLYGON ((2.20055 48.71317, 2.20059 48.71316, ..."


# Add markers of the CartoModule to map

In [7]:
import ast
#load the receiver map for selected map and layer
marker_gdf = gpd.read_file(ModuleMapPath)
#transform the str defining the list in column coordonneesEPSG3857 into a python list
marker_gdf["coordonneesEPSG3857"] = marker_gdf["coordonneesEPSG3857"].apply(ast.literal_eval)
#transform the list of coordinates into points in geometry column
marker_gdf["geometry"] = marker_gdf["coordonneesEPSG3857"].apply(lambda coords: Point(coords))
#drop the column coordonneesEPSG3857 and the layer id
marker_gdf = marker_gdf.drop(columns=["coordonneesEPSG3857","idCouche"])
#set the crs
marker_gdf=marker_gdf.set_crs(crs="epsg:3857", allow_override=True)
# change the projection to lat/lon
marker_gdf=marker_gdf.to_crs(epsg=4326)
marker_gdf.head()




,macModule,geometry
0,A8032A31204E,POINT (2.20079 48.71316)
1,C45BBE39F42A,POINT (2.20110 48.71315)
2,C45BBE39F9AE,POINT (2.20120 48.71314)
3,A8032A311FAA,POINT (2.20072 48.71317)
4,C45BBE39F56A,POINT (2.20117 48.71318)


In [8]:
#to optimize the getting of location of macModule, we create a dictionnary
MacModuleLocation = marker_gdf.set_index("macModule").to_dict()["geometry"]
MacModuleLocation
#it display the approximation of point location in print but it is not the case in the dataframe

{'A8032A31204E': <POINT (2.201 48.713)>,
 'C45BBE39F42A': <POINT (2.201 48.713)>,
 'C45BBE39F9AE': <POINT (2.201 48.713)>,
 'A8032A311FAA': <POINT (2.201 48.713)>,
 'C45BBE39F56A': <POINT (2.201 48.713)>,
 'C45BBE37B346': <POINT (2.201 48.713)>,
 'C45BBE386952': <POINT (2.201 48.713)>,
 'C45BBE386362': <POINT (2.201 48.713)>,
 'C45BBE384CD2': <POINT (2.201 48.713)>,
 'C45BBE39870E': <POINT (2.201 48.713)>,
 'A8032A312052': <POINT (2.201 48.713)>,
 'A8032A31204A': <POINT (2.201 48.713)>,
 'A8032A311F6A': <POINT (2.201 48.713)>,
 'A8032A311F56': <POINT (2.201 48.713)>,
 'C45BBE38FEAA': <POINT (2.201 48.713)>,
 'A8032A30FBFE': <POINT (2.2 48.713)>,
 'A8032A311DAA': <POINT (2.201 48.713)>,
 'A8032A30FB9E': <POINT (2.2 48.713)>,
 'C45BBE39F4D6': <POINT (2.201 48.713)>,
 'A8032A311F96': <POINT (2.201 48.713)>,
 'A8032A311F66': <POINT (2.202 48.713)>,
 'A8032A311FBA': <POINT (2.202 48.713)>,
 'A8032A311F52': <POINT (2.201 48.713)>,
 'A8032A311DBA': <POINT (2.201 48.713)>,
 'A8032A30FBEE': <PO

### Bind the received RSSI Data to the CartoModule Marker on Map

In [9]:
#load the received rssi data
rssi_df = pd.read_csv(Data_Path)
#timestamp is a string, we need to convert it to datetime
rssi_df["timestamp"] = pd.to_datetime(rssi_df["timestamp"])
# set the timestamp as index
rssi_df=rssi_df.sort_values('timestamp')
#drop the duplicates
rssi_df=rssi_df.drop_duplicates()
rssi_df.index = pd.RangeIndex(len(rssi_df.index))


In [10]:
rssi_df['geometry']=rssi_df["macModule"].apply(lambda macModule: MacModuleLocation[macModule])
rssi_df.head()

,timestamp,macModule,rssi,geometry
0,2023-06-01 12:35:46.605,A8032A311DAA,-66,POINT (2.200573795873236 48.713174265469796)
1,2023-06-01 12:35:46.617,A8032A311F6A,-67,POINT (2.200638986440939 48.71317445007082)
2,2023-06-01 12:35:46.693,A8032A311F96,-71,POINT (2.2006869316821076 48.71319455060883)
3,2023-06-01 12:35:48.290,A8032A31204A,-78,POINT (2.2007622901244153 48.71318498422266)
4,2023-06-01 12:35:48.332,A8032A311F56,-70,POINT (2.200612881651723 48.713203074448835)


In [11]:
#create a list of timestamps between min_timestamp and max_timestamp with a step of sampling_time
min_timestamp=rssi_df.iloc[0]['timestamp']+pd.Timedelta(sampling_time, unit="ms")
max_timestamp=rssi_df.iloc[-1]['timestamp']
timestamp_list= pd.date_range(start=min_timestamp, end=max_timestamp, freq=str(sampling_time)+"ms").tolist()


In [12]:
def timed_marker_intensity_list(rssi_df, sampling_time):
    min_timestamp=rssi_df.iloc[0]['timestamp']+pd.Timedelta(sampling_time, unit="ms")
    max_timestamp=rssi_df.iloc[-1]['timestamp']
    timestamp_list= pd.date_range(start=min_timestamp, end=max_timestamp, freq=str(sampling_time)+"ms").tolist()
    #implementation with a list of dataframes ouput
    marker_intensity_gdf_list=[]
    sampling_time=pd.Timedelta(sampling_time, unit="ms")
    for current_timestamp in timestamp_list:
        #for each module, we keep only the rssi values between current_timestamp and current_timestamp-sampling_time
        rssi_df_current=rssi_df[(rssi_df["timestamp"] > current_timestamp-sampling_time)&(rssi_df["timestamp"] < current_timestamp)]
        rssi_df_current=rssi_df_current.drop(columns=["timestamp","macModule"]).groupby("macModule").mean()
        marker_intensity_gdf=marker_gdf.merge(rssi_df_current, left_on="macModule", right_on="macModule")
        #mean of the rssi values for each module
        marker_intensity_gdf_list.append(marker_intensity_gdf)
    return(marker_intensity_gdf_list)

In [13]:
def getMarkerDictList(rssi_df, sampling_time, min_timestamp, max_timestamp):
    MarkerDictList = []
    #i in rssi_df.index
    index_count=0
    max_count = len(rssi_df)-1
    timestamp_list=pd.date_range(start=min_timestamp, end=max_timestamp, freq=str(sampling_time)+"ms").tolist()
    #on parcours la liste des timestamp, on fait en sorte de profiter que les données sont triées par timestamp, pour cela on crée un dict avec les macModule en clé et les rssi en valeur pour chaque intervalle de temps
    for timestamp in timestamp_list:
        markerdict={}
        while index_count!= max_count and rssi_df.iloc[index_count]['timestamp'] < timestamp:
            MacModule=rssi_df.iloc[index_count]['macModule']
            #if key does not exist, we create it and add the rssi value
            if MacModule not in markerdict:
                markerdict[MacModule]=[rssi_df.iloc[index_count]['rssi']]
            #if key exist, we add the rssi value
            else:
        
                markerdict[MacModule].append(rssi_df.iloc[index_count]['rssi'])    
            index_count+=1
        MarkerDictList.append(markerdict)
    return(MarkerDictList)

In [14]:
MarkerDictList=getMarkerDictList(rssi_df, sampling_time, min_timestamp, max_timestamp )

In [15]:

#todo : display the signal intensity on the map must create different feature for each different display change ( green, orange, red, )
def MarkerLayer(MarkerDictList):
    timestamp_list=pd.date_range(start=min_timestamp, end=max_timestamp, freq=str(sampling_time)+"ms").tolist()
    input_format = '%Y-%m-%d %H:%M:%S.%f'
    output_format = '%Y-%m-%dT%H:%M:%S'
    
    pointList=[]
    MarkerTimestamplist=[]
    rssiList=[]
    for index,timestamp in enumerate(timestamp_list):
        dt = datetime.strptime(str(timestamp), input_format)
        new_timestamp = dt.strftime(output_format)

        for MarkerDict in MarkerDictList[index]:
                point=MacModuleLocation[MarkerDict]
                pointList.append([point.x,point.y])
                MarkerTimestamplist.append(new_timestamp)
                rssiList.append(str(MarkerDictList[index][MarkerDict]))
    print(pointList)
    print(MarkerTimestamplist)
    print(rssiList)
    
    features = [{
        'type': 'Feature',
        'geometry': {
            'type': 'MultiPoint',
            'coordinates': pointList
        },
        'properties': {
            'times': MarkerTimestamplist,
            "icon": "circle",
            "popup": rssiList,
            "iconstyle": {
                "fillColor": "red"                        
            }}
        }]
        # pp = folium.Popup("rssi = ",str(MarkerDictList[index][MarkerDict]))
        # feature.add_child(pp)
    print(features)
    MarkerLayer=plugins.TimestampedGeoJson(
                        {"type": "FeatureCollection","features": features},
                        period='PT2S',
                        duration='PT1S',
                       transition_time=sampling_time,
                       auto_play=True)
    return(MarkerLayer)

In [16]:

#issue: display only the one point per timestamp, should redo the way of doing the feature  (list of point and list of timestamp)
def TestMarkerLayer():
    features=[]
    timestamp_list=pd.date_range(start=min_timestamp, end=max_timestamp, freq=str(sampling_time)+"ms").tolist()
    input_format = '%Y-%m-%d %H:%M:%S.%f'
    output_format = '%Y-%m-%dT%H:%M:%S'
    
    
    features = [{
        'type': 'Feature',
        'geometry': {
            'type': 'MultiPoint',
            'coordinates': [[2.2007200581389768,48.713166904690254]]
        },
        'properties': {
            'times': ["2023-06-01T12:35:46"],
            "icon": "circle",
            "iconstyle": {
                "fillColor": "red",
                "radius": 100                     
            }}
    }]

    MarkerLayer=plugins.TimestampedGeoJson(
                        {"type": "FeatureCollection","features": features},
                        period='PT2S',
                       transition_time=sampling_time,
                       auto_play=True)
    return(MarkerLayer)

In [17]:
m = folium.Map(location = [48.713166904690254, 2.2007200581389768], tiles="Openstreetmap",max_zoom=30, zoom_start=20,crs="EPSG3857")
tooltip = folium.features.GeoJsonTooltip(fields=['ID_element', 'nom'], 
                                        labels=True,
                                        stick=False)
folium.GeoJson(map_gdf, tooltip=tooltip).add_to(m)




In [18]:
marker_layer= MarkerLayer(MarkerDictList)
print(type(marker_layer))
marker_layer.add_to(m)


[[2.200573795873236, 48.713174265469796], [2.200638986440939, 48.71317445007082], [2.2006869316821076, 48.71319455060883], [2.2007622901244153, 48.71318498422266], [2.200612881651723, 48.713203074448835], [2.2007200581389768, 48.713166904690254], [2.200573795873236, 48.713174265469796], [2.200612881651723, 48.713203074448835], [2.2007200581389768, 48.713166904690254], [2.200638986440939, 48.71317445007082], [2.2006869316821076, 48.71319455060883], [2.200612881651723, 48.713203074448835], [2.200638986440939, 48.71317445007082], [2.2006869316821076, 48.71319455060883], [2.2007866061683496, 48.713157751163116], [2.2007622901244153, 48.71318498422266], [2.200612881651723, 48.713203074448835], [2.200573795873236, 48.713174265469796], [2.2007866061683496, 48.713157751163116], [2.200638986440939, 48.71317445007082], [2.2006869316821076, 48.71319455060883], [2.2007866061683496, 48.713157751163116], [2.2007622901244153, 48.71318498422266], [2.200612881651723, 48.713203074448835], [2.20072005813

In [19]:
folium.CircleMarker([48.713166904690254, 2.2007200581389768], radius=7, color='#FF0000', fill_color='#0080bb').add_to(m)

In [20]:
folium.LayerControl().add_to(m)

In [21]:
m.save('output/marker_map.html')

In [22]:
m

# must redo the plugin: leaflet code :

"""
function addGeoJSONLayer(map, data) {
    var icon = L.icon({
        iconUrl: 'img/bus.png',
        iconSize: [22, 22],
        iconAnchor: [11, 11]
    });

    var geoJSONLayer = L.geoJSON(data, {
        pointToLayer: function (feature, latLng) {
            if (feature.properties.hasOwnProperty('last')) {
                return new L.Marker(latLng, {
                    icon: icon
                });
            }
            return L.circleMarker(latLng);
        }
    });

    var geoJSONTDLayer = L.timeDimension.layer.geoJson(geoJSONLayer, {
        updateTimeDimension: true,
        duration: 'PT2M',
        updateTimeDimensionMode: 'replace',
        addlastPoint: true
    });

    // Show both layers: the geoJSON layer to show the whole track
    // and the timedimension layer to show the movement of the bus
    geoJSONLayer.addTo(map);
    geoJSONTDLayer.addTo(map);
}

var map = L.map('map', {
    zoom: 14,
    fullscreenControl: true,
    timeDimensionControl: true,
    timeDimensionControlOptions: {
        timeSliderDragUpdate: true,
        loopButton: true,
        autoPlay: true,
        playerOptions: {
            transitionTime: 1000,
            loop: true
        }
    },
    timeDimension: true,
    center: [36.72, -4.43]
});

var osmLayer = L.tileLayer('https://{s}.tile.osm.org/{z}/{x}/{y}.png', {
    attribution: '&copy; <a href="https://osm.org/copyright">OpenStreetMap</a> contributors'
});
osmLayer.addTo(map);

var oReq = new XMLHttpRequest();
oReq.addEventListener("load", (function (xhr) {
    var response = xhr.currentTarget.response;
    var data = JSON.parse(response);
    addGeoJSONLayer(map, data);
}));
oReq.open('GET', 'data/track_bus699.geojson');
oReq.send();
"""

In [23]:
# Function to draw points in the map
def draw_points(map_object, list_of_points, layer_name, line_color, fill_color, text):

    fg = folium.FeatureGroup(name=layer_name)

    for point in list_of_points:
        fg.add_child(
            folium.CircleMarker(
                point,
                radius=1,
                color=line_color,
                fill_color=fill_color,
                popup=(folium.Popup(text)),
            )
        )

    map_object.add_child(fg)

# implementation of low precision localisation

### 1. Naive implementation 

we're looking for 3 points at each sample,
if no 3 receivers received , we take the last 3 received,
if more than 3 receivers received we keep the 3 highest RSSI signal received

In [24]:
def max_marker_dict(marker_dict,edge_number=3):
    #keep only the edge_number modules with the highest rssi values
    #if there is less than edge_number modules, we keep all of them

    if len(marker_dict) > edge_number:
        # to do : function only if number of received signal for the module in list is 1 
        marker_dict={key: marker_dict[key] for key in sorted(marker_dict, key=marker_dict.get, reverse=True)[:edge_number]}
    return(marker_dict)

#wprks with polygon with moe than 3 points but not with 3 points because it is not a convex polygon, should use convex hull
list_of_list_of_points=[]
min_edge_number=3
max_edge_number=3
for i in range(len(MarkerDictList)):
    marker_dict=MarkerDictList[i]
    list_of_points = []
    marker_dict=max_marker_dict(marker_dict,max_edge_number)
    for mac_module,rssi in marker_dict.items():
        list_of_points.append([MacModuleLocation[mac_module].x,MacModuleLocation[mac_module].y])
    
    count_backward=1
    while len(list_of_points) < min_edge_number and i-count_backward >= 0:

        points_to_add=3-len(list_of_points)
        marker_dict=MarkerDictList[i-count_backward]
        marker_dict=max_marker_dict(marker_dict,points_to_add)

        
        for mac_module,rssi in marker_dict.items():
            location=[MacModuleLocation[mac_module].x,MacModuleLocation[mac_module].y]
            if location in list_of_points:
                continue
            list_of_points.append(location)
        count_backward+=1    

        
    list_of_list_of_points.append(list_of_points)
#eject the marker not having the right number of points ( not triangle for example)
for index,list1 in enumerate(list_of_list_of_points):
    if len(list1)<min_edge_number:
        print("error")
        list_of_list_of_points.pop(index)


error


list_of_list_of_points est une liste définissant les points associés aux 3 récepteurs les plus forts pour chaque temps. on cherche maintenant à afficher les polygones associés dans une map temporelle

In [25]:

from folium.plugins import TimestampedGeoJson
from datetime import datetime

input_format = '%Y-%m-%d %H:%M:%S.%f'
output_format = '%Y-%m-%dT%H:%M:%S'

def plot_map_and_polygonal_localization(list_of_list_of_points):

    # Create the map used to visualise the locations of the images.

    m = folium.Map(location = [48.713166904690254, 2.2007200581389768], tiles="Openstreetmap", zoom_start=22,max_zoom=28,crs="EPSG3857")
    folium.GeoJson(map_gdf, tooltip=tooltip).add_to(m)
    #folium.GeoJson(marker_gdf, tooltip=tooltip2).add_to(m)
    features = []
    timestamp_list=[]
    for index in range(len(list_of_list_of_points)):
        timestamp = min_timestamp+index*pd.Timedelta(sampling_time, unit="ms")
        dt = datetime.strptime(str(timestamp), input_format)
        new_timestamp = dt.strftime(output_format)
        timestamp_list.append(new_timestamp)
    print(timestamp_list)
    print(list_of_list_of_points)
    print(len(list_of_list_of_points),len(timestamp_list))

    features = [{
        'type': 'Feature',
        'geometry': {
            'type': 'Polygon',
            'coordinates': list_of_list_of_points
        },
        'properties': {
            'times': timestamp_list,
            'style': {'color': 'red', 'weight': 1},
            
        }
    }]

    TimestampedGeoJson(features,
                        period='PT2S',
                        duration='PT1S',
                       transition_time=sampling_time,  
                       auto_play=True).add_to(m)
    folium.LayerControl().add_to(m)

    m.save('map_with_gps_locations.html')
    return(m)


In [26]:
plot_map_and_polygonal_localization(list_of_list_of_points)

['2023-06-01T12:35:48', '2023-06-01T12:35:50', '2023-06-01T12:35:52', '2023-06-01T12:35:54', '2023-06-01T12:35:56', '2023-06-01T12:35:58', '2023-06-01T12:36:00', '2023-06-01T12:36:02', '2023-06-01T12:36:04', '2023-06-01T12:36:06', '2023-06-01T12:36:08', '2023-06-01T12:36:10', '2023-06-01T12:36:12', '2023-06-01T12:36:14', '2023-06-01T12:36:16', '2023-06-01T12:36:18', '2023-06-01T12:36:20', '2023-06-01T12:36:22', '2023-06-01T12:36:24', '2023-06-01T12:36:26', '2023-06-01T12:36:28', '2023-06-01T12:36:30', '2023-06-01T12:36:32', '2023-06-01T12:36:34', '2023-06-01T12:36:36', '2023-06-01T12:36:38', '2023-06-01T12:36:40', '2023-06-01T12:36:42', '2023-06-01T12:36:44', '2023-06-01T12:36:46', '2023-06-01T12:36:48', '2023-06-01T12:36:50', '2023-06-01T12:36:52', '2023-06-01T12:36:54', '2023-06-01T12:36:56', '2023-06-01T12:36:58', '2023-06-01T12:37:00', '2023-06-01T12:37:02', '2023-06-01T12:37:04', '2023-06-01T12:37:06', '2023-06-01T12:37:08', '2023-06-01T12:37:10', '2023-06-01T12:37:12', '2023-06-0

In [27]:
print(list_of_list_of_points)

[[[2.200573795873236, 48.713174265469796], [2.200638986440939, 48.71317445007082], [2.200612881651723, 48.713203074448835]], [[2.200638986440939, 48.71317445007082], [2.200612881651723, 48.713203074448835], [2.2007200581389768, 48.713166904690254]], [[2.200612881651723, 48.713203074448835], [2.200638986440939, 48.71317445007082], [2.200573795873236, 48.713174265469796]], [[2.200612881651723, 48.713203074448835], [2.200638986440939, 48.71317445007082], [2.2006869316821076, 48.71319455060883]], [[2.200573795873236, 48.713174265469796], [2.2007866061683496, 48.713157751163116], [2.200612881651723, 48.713203074448835]], [[2.200638986440939, 48.71317445007082], [2.2006869316821076, 48.71319455060883], [2.2007866061683496, 48.713157751163116]], [[2.200612881651723, 48.713203074448835], [2.200638986440939, 48.71317445007082], [2.2007200581389768, 48.713166904690254]], [[2.2007866061683496, 48.713157751163116], [2.2007200581389768, 48.713166904690254], [2.2007622901244153, 48.71318498422266]],

In [28]:
def polygonlist_to_polygoncenterlist(list_of_list_of_points):
    centerList=[]
    for list1 in list_of_list_of_points:
        n=0
        xlist=0
        ylist=0
        for point in list1:    
            xlist+=point[0]
            ylist+=point[1]
            n+=1
        centerList.append([xlist/n,ylist/n])
    return(centerList)
centerList=polygonlist_to_polygoncenterlist(list_of_list_of_points)
        

In [29]:
centerList
for _center in centerList:
    if len(_center) != 2:
        print("error")

In [30]:

from folium.plugins import TimestampedGeoJson
from datetime import datetime

input_format = '%Y-%m-%d %H:%M:%S.%f'
output_format = '%Y-%m-%dT%H:%M:%S'

timestamp_list=[]
for index in range(len(list_of_list_of_points)):
    timestamp = min_timestamp+index*pd.Timedelta(sampling_time, unit="ms")
    dt = datetime.strptime(str(timestamp), input_format)
    new_timestamp = dt.strftime(output_format)
    timestamp_list.append(new_timestamp)
        
m = folium.Map(location = [48.713166904690254, 2.2007200581389768], tiles="Openstreetmap", zoom_start=22,max_zoom=28,crs="EPSG3857")
folium.GeoJson(map_gdf, tooltip=tooltip).add_to(m)
#folium.GeoJson(marker_gdf, tooltip=tooltip2).add_to(m)
print(centerList)
print(timestamp_list)
print(len(centerList),len(timestamp_list))

features = [{
    'type': 'Feature',
    'geometry': {
        'type': 'MultiPoint',
        'coordinates': centerList
    },
    'properties': {
        'times': timestamp_list,
        'style': {'color': 'red', 'weight': 1},
        
    }
}]

TimestampedGeoJson(features,
                    period='PT2S',
                    duration='PT1S',
                    transition_time=sampling_time,  
                    auto_play=True).add_to(m)
folium.LayerControl().add_to(m)

m.save('map_with_point_locations.html')
m


[[2.2006085546553, 48.71318392999648], [2.2006573087438794, 48.7131814764033], [2.2006085546552994, 48.71318392999648], [2.20064626659159, 48.71319069170949], [2.200657761231103, 48.71317836369391], [2.2007041747637985, 48.71317558394759], [2.2006573087438794, 48.7131814764033], [2.200756318143914, 48.71316988002534], [2.2006332379987605, 48.71318108871648], [2.200657761231103, 48.71317836369391], [2.200681992087341, 48.713178635123306], [2.200698409971705, 48.71318498778725], [2.2007041747637985, 48.71317558394759], [2.2007230933151667, 48.71318214650725], [2.2007311986631444, 48.71317306882074], [2.200811704926871, 48.713163695028776], [2.2008725645624536, 48.713157206483885], [2.200822374297332, 48.713171012655344], [2.2007230933151667, 48.71318214650725], [2.2007563181439136, 48.71316988002534], [2.2007816294088762, 48.71316725825097], [2.2007230933151667, 48.71318214650724], [2.2007626541458216, 48.71318585934466], [2.200805352482504, 48.7131640056218], [2.200897684043223, 48.7131

In [31]:

import numpy as np
number_of_points=len(centerList)
longitude_data=np.empty(number_of_points)
latitude_data=np.empty(number_of_points)
for index,point in enumerate(centerList):
    longitude_data[index]=point[0]
    latitude_data[index]=point[1]

In [32]:
def kalman_filter(data, Q, R):
    n = len(data)
    x_est = np.zeros(n)
    P = np.zeros(n)
    x_est[0] = data[0]
    P[0] = 1.0

    for i in range(1, n):
        x_pred = x_est[i-1]
        P_pred = P[i-1] + Q

        K = P_pred / (P_pred + R)
        x_est[i] = x_pred + K * (data[i] - x_pred)
        print("diff=",x_est[i]-x_pred)
        P[i] = (1 - K) * P_pred

    return x_est

Q = 30  # Process noise
R = 10  # Measurement noise


In [33]:

longitude_filtered = kalman_filter(longitude_data, Q, R)
latitude_filtered = kalman_filter(latitude_data, Q, R)
centerList_filtered=np.vstack((longitude_data,latitude_data)).T.tolist()
print(centerList)
print(centerList_filtered)

diff= 3.686284746251545e-05
diff= -2.9112197483360802e-05
diff= 2.3705985635835702e-05
diff= 1.4045259934647447e-05
diff= 3.9657926444558456e-05
diff= -2.88074142269501e-05
diff= 7.233247737703508e-05
diff= -8.229515605195914e-05
diff= 2.2289365704253328e-06
diff= 1.963878822452969e-05
diff= 1.7090126141816597e-05
diff= 8.128526940875958e-06
diff= 1.666654214416141e-05
diff= 9.892173240544366e-06
diff= 6.576824490256428e-05
diff= 6.188412199970017e-05
diff= -2.6798978542963425e-05
diff= -8.415310717024838e-05
diff= 8.726627083177618e-06
diff= 2.1849849491939466e-05
diff= -4.175857047128417e-05
diff= 2.2588483404195614e-05
diff= 3.850116591763708e-05
diff= 8.109650314880312e-05
diff= -3.9969185221089276e-05
diff= -3.200415736426265e-05
diff= 7.798939847702968e-05
diff= 1.1150424938932701e-07
diff= -3.1066726867035044e-05
diff= -4.389722834696741e-05
diff= -9.161885017761051e-06
diff= 1.9335197186443054e-05
diff= 6.450199376839549e-06
diff= 5.345189598937594e-06
diff= 4.19577728112408e-0

In [34]:

input_format = '%Y-%m-%d %H:%M:%S.%f'
output_format = '%Y-%m-%dT%H:%M:%S'

timestamp_list=[]
for index in range(len(list_of_list_of_points)):
    timestamp = min_timestamp+index*pd.Timedelta(sampling_time, unit="ms")
    dt = datetime.strptime(str(timestamp), input_format)
    new_timestamp = dt.strftime(output_format)
    timestamp_list.append(new_timestamp)
        
m = folium.Map(location = [48.713166904690254, 2.2007200581389768], tiles="Openstreetmap", zoom_start=22,max_zoom=28,crs="EPSG3857")
folium.GeoJson(map_gdf, tooltip=tooltip).add_to(m)
#folium.GeoJson(marker_gdf, tooltip=tooltip2).add_to(m)
print(centerList_filtered)
print(timestamp_list)
print(len(centerList_filtered),len(timestamp_list))

features = [{
    'type': 'Feature',
    'geometry': {
        'type': 'MultiPoint',
        'coordinates': centerList_filtered
    },
    'properties': {
        'times': timestamp_list,
        'style': {'color': 'red', 'weight': 1},
        
    }
}]

TimestampedGeoJson(features,
                    period='PT2S',
                    duration='PT1S',
                    transition_time=sampling_time,  
                    auto_play=True).add_to(m)
folium.LayerControl().add_to(m)

m.save('map_with_point_locations_filtered.html')

[[2.2006085546553, 48.71318392999648], [2.2006573087438794, 48.7131814764033], [2.2006085546552994, 48.71318392999648], [2.20064626659159, 48.71319069170949], [2.200657761231103, 48.71317836369391], [2.2007041747637985, 48.71317558394759], [2.2006573087438794, 48.7131814764033], [2.200756318143914, 48.71316988002534], [2.2006332379987605, 48.71318108871648], [2.200657761231103, 48.71317836369391], [2.200681992087341, 48.713178635123306], [2.200698409971705, 48.71318498778725], [2.2007041747637985, 48.71317558394759], [2.2007230933151667, 48.71318214650725], [2.2007311986631444, 48.71317306882074], [2.200811704926871, 48.713163695028776], [2.2008725645624536, 48.713157206483885], [2.200822374297332, 48.713171012655344], [2.2007230933151667, 48.71318214650725], [2.2007563181439136, 48.71316988002534], [2.2007816294088762, 48.71316725825097], [2.2007230933151667, 48.71318214650724], [2.2007626541458216, 48.71318585934466], [2.200805352482504, 48.7131640056218], [2.200897684043223, 48.7131

In [35]:
m